# Prediction Algorithms

The main idea is to predict the next day low, upper, open and close to determine a strategy for the next day

In [1]:
import pandas as pd
from datetime import datetime
from datetime import date 
import numpy as np

from time import sleep
from tqdm.notebook import tqdm, trange

import seaborn as sns
import matplotlib as plt
import plotly.graph_objects as go

from tqdm.notebook import tqdm
import warnings
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split as tts
from xgboost import XGBRegressor as XGBR
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.ensemble import GradientBoostingRegressor as GBR

# Functions

In [2]:
#Conversion from strings to numeric
def strnum(x):
    try:
        x=x.replace(',','')
        return float(x)
    except:
        return x

#-----------movingAvg function-----------------
#Calculates the Moving Average

#df - list with the dataframes
#x - index of the coin, e.g. 0- BTC, 1-ETH... of the databases' list who contains the collections of data
#dname - name of the destination column of the moving average calculation (not creared yet)
#oname - name of the origin column of the data to take the information for the moving avarege
#msize - number of registers to calculate the moving average

def movingAvg(df,x,dname,oname,msize):
    df[x][dname] = 0.0
    for i in range(len(df[x])):
        temp=0.0
        if i<msize:
            pass
        else:
            for j in range(msize):
                temp+= data[x][oname][i-msize+j]
            df[x][dname][i]=temp/msize
#-----------------oheDate--------------------
#Makes one hot econfing to date depending day of the week, day of the month, week of year and month of year

def oheDate(df,x):
    df[x]['weekday']=df[x]['date']
    df[x]['weekday']=df[x]['weekday'].apply(lambda z: str(z.weekday()))
    
    #df[x]['calendarweek']=df[x]['date']
    #df[x]['calendarweek']=df[x]['date'].apply(lambda z: str(z.isocalendar()[1]))
    
    #df[x]['dayofmonth']=df[x]['date']
    #df[x]['dayofmonth']=df[x]['date'].apply(lambda z: str(z.day))
    
    #df[x]['month']=df[x]['date']
    #df[x]['month']=df[x]['date'].apply(lambda z: str(z.month))
    
    #Contanating dummies
    ohe=pd.get_dummies(df[x][['weekday']])#,'calendarweek']])#,'dayofmonth','month']])
    df[x]=pd.concat([df[x],ohe],axis=1)
    #Eliminating categorical columns
    df[x].drop(columns=['weekday'], inplace=True)
    
    
#--------------------yesterday-----------------
#Takes n "days" to the current day as new column (parameter) for the prediction
def yesterday(df,x,days=1,closev=True,openv=True,highv=True,lowv=True,volumev=True):
    
    #Values selector from the function inputs
    values_names=['close','open','high','low','volume']
    values=[closev,openv,highv,lowv,volumev]
    vals=[]
    for i,e in enumerate(values):
        if e==True:
            vals.append(values_names[i])
    yestvals=[]#List of added variables
    
    #Main set    
    for day in range(days):
        for val in vals:
            y=val+'_'+str(day)
            df[x]['yesterday_'+y]=0.0
            yestvals.append('yesterday_'+y)
            for i in range(len(df[x])-1):
                df[x]['yesterday_'+y][i+1]=df[x][val][i]
    
    return yestvals

# Load and Transform

In [58]:
#Global Variables
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']

#Opening All databases
data=[]
for i,e in enumerate(coins):
    file=e+'-mxn.csv'
    data.append(pd.read_csv(file))
#Conversion of dates from strin to timestamps
for i in range(len(coins)):
    data[i].date=data[i].date.apply(lambda x: datetime.strptime(x,'%Y-%m-%d'))

# Conversion to numeric 
for i in range(len(coins)):
    data[i].open=data[i].open.apply(lambda x: strnum(x))
    data[i].close=data[i].close.apply(lambda x: strnum(x))
    data[i].low=data[i].low.apply(lambda x: strnum(x))
    data[i].high=data[i].high.apply(lambda x: strnum(x))
    data[i].volume=data[i].volume.apply(lambda x: strnum(x))

#MACD Calculation
metricas=['open','close','high','low','volume']

for i in tqdm(range(len(coins))):
    for m in metricas:#tqdm(metricas):
        movingAvg(data,i,'PMEF'+ m,m,12)
        movingAvg(data,i,'PMES'+m,m,26)
        data[i]['MACD'+m]=data[i]['PMEF'+m]-data[i]['PMES'+m]
        movingAvg(data,i,'MACD9'+m,'MACD'+m,9)
        data[i]['diffMACD'+m]=data[i]['MACD'+m]-data[i]['MACD9'+m]
    oheDate(data,i)
    yv=yesterday(data,i,14)



# Predictior Function

In [59]:
def predictor(data=data,end=0):
    #Create Dataframe for predictions
    preds= pd.DataFrame(columns=metricas)
    preds['coin'] =""
    cols=preds.columns.tolist()
    cols=cols[-1:]+cols[:-1]
    preds=preds[cols]
    preds

    for x in tqdm(range(len(coins))):
        test=data[x][35:len(data[x])-1-end] #35 is 26 + 9 for all the number of samples who doesn't has Moving Avg  
        dc=['date','open','close','low','high','volume']
        timevars = [ 'weekday_'+ str(x) for x in range(7)]

        #Adding new row for predictions
        preds=preds.append([0], ignore_index=True).drop(0,axis=1)
        preds=preds[cols]
        preds['coin'][x]=coins[x]

        for m in metricas:
            #Predictions
            X_train, X_test, y_train, y_test=tts(test[['PMEF'+m,'PMES'+m,'MACD'+m,'MACD9'+m]+timevars+yv],test[m], shuffle=True,random_state=100,test_size=.1)
            modelo=XGBR()
            modelo.fit(X_train, y_train)
            preds[m][x]=modelo.predict(data[x][len(data[x])-1-end:len(data[x])-end][['PMEF'+m,'PMES'+m,'MACD'+m,'MACD9'+m]+timevars+yv])[0]
            #train_score=modelo.score(X_train, y_train)
            #test_score=modelo.score(X_test, y_test)
            #print ('Score para', coins[x],m,train_score, test_score)

    #Getting oppenings to make initial corrections

    preds['ropen']=0.0
    for i,e in enumerate(coins):
        preds.ropen[i]=data[i]['open'][len(data[i])-1-end]

    preds['corr']=preds.ropen - preds.open
    preds['pcorr']= (1 - preds.open/preds.ropen)*100
    preds['pcorr']= preds['pcorr'].apply(lambda x: str(x.round(1))+'%')


    #Calculation of investment factors

    preds['go']= preds.close - preds.open
    preds['go']= preds['go'].apply(lambda x: 1 if x>0 else 0)
    preds['buy']=preds.open - (preds.open-preds.low)/3 # 1/3 below the opening regarding the lowest point
    preds['sell']=(preds.high-preds.close)/3+preds.close # 1/3 above the closure regarding the highes point
    preds['rate']=(preds.sell/preds.buy-1)*100
    preds.go=preds.go*preds.rate
    preds.go=preds['go'].apply(lambda x: 1 if x>0 else 0)
    preds.rate=preds.rate.apply(lambda x: x.round(1))

    #Calculuation correction

    preds['buya'] = preds['buy'] + preds['corr']
    preds['sella'] = preds['sell'] # + preds['corr']
    preds['rrate']= (preds['sella']/preds['buya'] - 1)*100
    preds['stoploss'] = preds['low'] + preds['corr']
    preds.rrate=preds.rrate.apply(lambda x: x.round(1))
    preds['go']=preds['rrate']
    preds['go']= preds['go'].apply(lambda x: 1 if x>2 else 0)
    
    return preds


In [60]:
preds=predictor()

# Display Predictions

In [61]:
sortedcols=['coin','buya','sella','stoploss','go','rrate']
preds[sortedcols]

,coin,buya,sella,stoploss,go,rrate
0,btc,261994,260164,254906,0,-0.7
1,eth,8700.13,8921.91,8431.62,1,2.5
2,xrp,6.63197,6.78211,6.3159,1,2.3
3,ltc,1290.09,1280.64,1258.6,0,-0.7
4,bch,6620.1,6903.44,6370.7,1,4.3
5,tusd,22.3786,22.488,22.2758,0,0.5
6,mana,2.18849,2.29651,2.16548,1,4.9
7,gnt,1.68831,1.81428,1.56493,1,7.5
8,bat,6.20313,6.27263,6.02939,0,1.1
9,dai,22.6866,22.8127,22.4999,0,0.6


In [62]:
preds

,coin,open,close,high,low,volume,ropen,corr,pcorr,go,buy,sell,rate,buya,sella,rrate,stoploss
0,btc,267745,256015,268462,257113,165.616,265538.03,-2206.5,-0.8%,0,264201,260164,-1.5,261994,260164,-0.7,254906
1,eth,8858.15,8894.92,8975.87,8455.39,1281.64,8834.38,-23.7665,-0.3%,1,8723.89,8921.91,2.3,8700.13,8921.91,2.5,8431.62
2,xrp,6.81365,6.64777,7.05078,6.33955,1.08908e+07,6.79,-0.0236487,-0.3%,1,6.65562,6.78211,1.9,6.63197,6.78211,2.3,6.3159
3,ltc,1309.01,1253.27,1335.37,1261.78,2163.84,1305.83,-3.17793,-0.2%,0,1293.26,1280.64,-1.0,1290.09,1280.64,-0.7,1258.6
4,bch,6811.54,6964.94,6780.44,6437.44,634.518,6744.80,-66.7376,-1.0%,1,6686.84,6903.44,3.2,6620.1,6903.44,4.3,6370.7
5,tusd,22.3841,22.4127,22.6385,22.2299,91186.2,22.43,0.0459,0.2%,0,22.3327,22.488,0.7,22.3786,22.488,0.5,22.2758
6,mana,2.182,2.20961,2.47031,2.14748,1.17579e+07,2.20,0.0180003,0.8%,1,2.17049,2.29651,5.8,2.18849,2.29651,4.9,2.16548
7,gnt,1.80088,1.79674,1.84935,1.61581,1.17709e+06,1.75,-0.0508804,-2.9%,1,1.73919,1.81428,4.3,1.68831,1.81428,7.5,1.56493
8,bat,6.25585,6.16445,6.48899,5.99524,258930,6.29,0.0341516,0.5%,0,6.16898,6.27263,1.7,6.20313,6.27263,1.1,6.02939
9,dai,22.9062,22.7389,22.9605,22.6262,20223.5,22.78,-0.126239,-0.6%,0,22.8129,22.8127,-0.0,22.6866,22.8127,0.6,22.4999


# Testing to find parameter to buy and sell

In [18]:
def check(n=1):
    
    metricas=['open','close','high','low','volume']
    diffs=[]
    
    for i in tqdm(range(n)):
        preds=predictor(data,i) #Getting the preductions of day i
        
        for c in range(10):
            for m in metricas:
                real=data[c][len(data[c])-i-1:len(data[c])-i].iloc[0][m]
                pred=(preds.iloc[c][m]).round(2)
                error=1-pred/real
                diffs.append([c,m,error.round(2),real,pred])
    return pd.DataFrame(diffs)

In [19]:
z=check(62)

All but volume has less than 2% of error in the prediction. Error in volume is huge

In [29]:
coins=['btc','eth','xrp','ltc','bch','tusd','mana','gnt','bat','dai']
metricas=['open','close','high','low','volume']
for i,e in enumerate(coins):
    for m in metricas:
        print(e,'in',m)
        print('min'   ,z[(z[0]==i) & (z[1]==m)][2].min().round(2)*100)
        print('max'   ,z[(z[0]==i) & (z[1]==m)][2].max().round(2)*100)
        print('mean'  ,z[(z[0]==i) & (z[1]==m)][2].mean().round(4)*100) 
        print('stdev' ,z[(z[0]==i) & (z[1]==m)][2].std().round(4)*100)
        print('')
    print ('---------------------------------')

btc in open
min -5.0
max 4.0
mean -0.02
stdev 1.05

btc in close
min -4.0
max 8.0
mean 0.98
stdev 2.87

btc in high
min -11.0
max 7.000000000000001
mean -0.21
stdev 2.78

btc in low
min -4.0
max 9.0
mean 1.06
stdev 2.34

btc in volume
min -449.0
max 72.0
mean -30.0
stdev 81.31

---------------------------------
eth in open
min -2.0
max 2.0
mean 0.06
stdev 0.9400000000000001

eth in close
min -8.0
max 8.0
mean 0.5599999999999999
stdev 3.32

eth in high
min -7.000000000000001
max 14.000000000000002
mean -0.19
stdev 3.3099999999999996

eth in low
min -4.0
max 8.0
mean 0.98
stdev 2.6599999999999997

eth in volume
min -488.0
max 78.0
mean -44.84
stdev 98.99

---------------------------------
xrp in open
min -2.0
max 2.0
mean -0.05
stdev 0.7100000000000001

xrp in close
min -12.0
max 10.0
mean 0.26
stdev 3.58

xrp in high
min -12.0
max 14.000000000000002
mean -0.02
stdev 3.75

xrp in low
min -5.0
max 6.0
mean 0.97
stdev 2.1999999999999997

xrp in volume
min -247.00000000000003
max 47.0
mean 

In [30]:
z[(z[0]==0) & (z[1]=='open')].describe()

,0,2,3,4
count,62.0,62.000000,62.000000,62.000000
mean,0.0,-0.000161,215557.186129,215557.234375
std,0.0,0.010479,14144.647842,13949.180729
min,0.0,-0.050000,202965.320000,201987.453125
25%,0.0,0.000000,207951.382500,207850.871094
50%,0.0,0.000000,210461.690000,210745.054688
75%,0.0,0.000000,213330.062500,212662.707031
max,0.0,0.040000,257000.000000,256686.296875


In [31]:
z

,0,1,2,3,4
0,0,open,-0.01,254154.970000,256686.296875
1,0,close,0.04,260362.650000,249308.921875
2,0,high,0.02,260960.170000,256135.218750
3,0,low,0.06,253167.080000,238844.703125
4,0,volume,-4.49,48.029895,263.850006
...,...,...,...,...,...
3095,9,open,0.00,21.990000,21.980000
3096,9,close,-0.02,21.670000,22.010000
3097,9,high,-0.03,22.220000,22.889999
3098,9,low,-0.00,21.650000,21.690001


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Columns: 108 entries, date to yesterday_volume_13
dtypes: datetime64[ns](1), float64(100), uint8(7)
memory usage: 81.3 KB


# Algorthms Intraday Trading

# Graphic

In [ ]:
#Graphic with Candle Sticks
i=0
fig=go.Figure(data=[go.Candlestick(x=data[i].date,
                                   open=data[i].open,
                                   high=data[i].high,
                                   low=data[i].low,
                                   close=data[i].close
                                  )
                   ]
             )
fig.show()

## Ideas

- Considerar acumulado de los volumenes para los pronósticos
- Considerar un criterio para la clasificacion de dias de compra, venta y compraventa
- Considadr pronosticos de más de un dia quizás de una semana para determinar puntos de compra venta dentro de la semana
- Reducir variables poco significativas
- Realizar visualizaciones para detectar correlaciones y variables poco significativas
- Hace clasificación no supervisada de 2 a 5 clusters